# Tutorial 02-02 - Working with ArcGIS Pro Maps

Let's go back to our work with highway data for GeoNinjas PythonAnalytics.  Now that our data is cleaned, one of our colleagues has asked with help creating a number of maps.  Rather than use a map series in ArcGIS pro, they've asked if we can use a template layout they've created and use Python to modify the map and export to PDF.

## Open the ArcGIS Pro Project

Although ArcGIS Pro projects can be comprised of a number of different files in a folder, when you reference them via arcpy we often will start with the **.aprx** file.  This is a good entry point that will let us access the data, maps, and layouts in the project.  You'll start this exercise by creating an **ArcGISProject** object using the .aprx file.

#### 1.  Import arcpy

In [ ]:
import arcpy

#### 2.  Define the path to the project

In [ ]:
# relative path from this script to the project
project_path = r".\Chapter 03 Files\Chapter 02 - Working with Maps.aprx"

#### 3.  Create an arcpy project object

In [ ]:
# creating a project object
project = arcpy.mp.ArcGISProject(project_path)

## Access the layout and elements

#### 1.  Access the layout you want to export

In this case, you happen to know that this project contains one layout that we want to modify.  Because of that, you can just accept the first layout returned from the `.listLayouts()` method.  If you had multiple layouts in the same project, we could use this method to search and/or iterate through the results.

In [ ]:
layout = project.listLayouts()[0]
layout.name

#### 2.  List elements in the layout

To access individual elements in the layout, you can call the `listElements()` method.  This will return any elements in the layout including text boxes, map frames, North arrows, charts, and more.  

In [ ]:
for element in layout.listElements():
    print(element.name)

#### 3.  Access specific elements in the layout

Since this is a fairly simple layout, you can just access the elements you want to modify by name.  In this case, you'll be doing some filtering in the *Map Frame* element and you'll change the text in the *Text* element.  You can access both of those by using list comprehensions.  Alternatively, you could just write a for-loop and iterate through all the elements until you find the one you want.

In [ ]:
title = [e for e in layout.listElements() if e.name == 'Text'][0]
map_frame = [e for e in layout.listElements() if e.name == 'Map Frame'][0]

## Fix Broken Layers

#### 1.  Identify broken layers

If you've already opened the ArcGIS Pro project in this folder, you may have noticed that there are two "broken" layers.  The "Counties" and "Highways" layers in the Map are referencing data sources with bad paths.  This happens less often than it used to in the days of ArcMap, but it's still something we can look for using arcpy.

In [ ]:
for layer in map_frame.map.listLayers():
    if layer.name in ['Highways_Intersect','Counties']:
        print(layer.name, layer.isBroken)

#### 2.  Change the data source to fix the layers

Not only can you identify this issue using arcpy, you can also fix it.  In this case, the data you need is in the project database.  You can iterate through each of the layers, get the connection properties, and modify those properties to read from the project database

In [ ]:
# iterate through each layer
for layer in map_frame.map.listLayers():
    
    # check only the layers we think are broken
    if layer.name in ['Highways_Intersect','Counties']:
        
        # get a copy the connection properties of the broken layer
        layer_conn_props = layer.connectionProperties
        
        # replace the database path in the copy of the connection properties
        layer_conn_props['connection_info']['database'] = project.databases[0]['databasePath']
        
        # use the copy we fixed to update the layer
        layer.updateConnectionProperties(
            layer.connectionProperties, layer_conn_props
        )
        print(layer.name, layer.isBroken)

Now that we've fixed our project, you can save your changes and move on.

In [ ]:
project.save()

## Work with the layers in the map view

#### 1.  Identify specific layers to work with

Use the `.listLayers()` method to search for specific layers

In [ ]:
# county layer
counties_layer = map_frame.map.listLayers('Counties')[0]

# identify the highways layer
highways_layer = map_frame.map.listLayers("Highways_Intersect")[0]

#### 2.  Filter the data in the map

Now that you've fixed the broken layers in the map, you can work on filtering the datasets.  In ArcGIS Pro, there's a concept of *definition queries* on layers.  These are SQL where clauses that can be used to filter the records or features that are shown in a layer.

You can iterate through both our highways and counties layers and set the definition query to use the *NAMELSAD* field to your county of interest.

In [ ]:
# pick a county to test with
county = 'Alameda County'

# set the definition query for the counties and highways layers
counties_layer.definitionQuery = f"NAMELSAD = '{county}'"
highways_layer.definitionQuery = f"NAMELSAD = '{county}'"

#### 3.  Change the zoom extent of the map

You'll also want to set the extent of our map to the county of interest.  This way you're zoomed to just the data that you're concerned about and not viewing the entire state.  You can get that extent by calling the `.getLayerExtent()` method of the MapFrame object (map_frame).  You can then set the MapFrame's extent to that layer extent.

In [ ]:
# get the county extent
county_extent = map_frame.getLayerExtent(counties_layer)

# set the MapFrame's extent
map_frame.camera.setExtent(county_extent)

## Modify layout and print to PDF

Now that you've filtered our data for a single county and set the extent for the map, you can export the map to a print-friendly PDF.  Before you do this, you can modify our map's title by working with the text element you identified in Step 2.

#### 1.  Set the title of the layout to the county name

In [ ]:
# set the title text
title.text = county

#### 2.  Export the layout to a PDF file

Now to export your layout to PDF, we can call the `.exportToPDF()` method on the layout object.

In [ ]:
# export the PDF.  Name the file using the county name.
layout.exportToPDF(f'./{county}.pdf')

## Repeat the process for multiple counties

Since you've worked out the logic for modifying the map and printing for one county, you can now reuse that code to export multiple counties.  

#### 1.  Define a list of counties to iterate through

In [ ]:
# list of counties to export
counties = [
    "Alameda County",
    "Marin County",
    "Contra Costa County",
    "San Francisco County",
    "Santa Clara County"
]

#### 2.  Repeat the export logic multiple times

Now go back and grab the operative portions of the code you wrote.  Put it in a "for" loop and repeat the process for multiple counties.  You can even define the file name of your exported pdfs by the county name as well.

In [ ]:
# iterate through the counties
for county in counties:
    # set the definition query for the counties and highways layers
    counties_layer.definitionQuery = f"NAMELSAD = '{county}'"
    highways_layer.definitionQuery = f"NAMELSAD = '{county}'"
    
    # get the county extent
    county_extent = map_frame.getLayerExtent(counties_layer)

    # set the MapFrame's extent
    map_frame.camera.setExtent(county_extent)
    
    # set the title text
    title.text = county
    
    # export the PDF.  Name the file using the county name.
    layout.exportToPDF(f'./{county}.pdf')